In [2]:
import pandas as pd
import requests

# Your OMDb API key
API_KEY = "d8fef20b"

# Input and output file names
INPUT_FILE = "C:/Users/nerea/Documents/MasterDTU/SocialGraphs_fall24/Projects/socialGraphs_fall24/FinalProject/csv/top_movie_data_with_similar.csv"  # Input CSV file
OUTPUT_FILE = "imdb_urls.csv"  # Output CSV file to store IMDb URLs

# Function to fetch IMDb URL from OMDb API
def get_imdb_url(movie_title, api_key):
    try:
        # Make the API request
        url = f"http://www.omdbapi.com/?t={movie_title}&apikey={api_key}"
        response = requests.get(url)
        data = response.json()
        
        # Check if the response contains an IMDb ID
        if data.get("Response") == "True" and "imdbID" in data:
            imdb_id = data["imdbID"]
            return f"https://www.imdb.com/title/{imdb_id}/"
        else:
            # Return None if no result is found
            return None
    except Exception as e:
        print(f"Error fetching data for {movie_title}: {e}")
        return None

# Read the input CSV file
try:
    # Load the CSV file
    df = pd.read_csv(INPUT_FILE)
    
    # Ensure the "name" column exists
    if "Name" not in df.columns:
        print("Error: The input CSV file must have a 'name' column.")
    else:
        # Fetch IMDb URLs for each movie title in the 'name' column
        df["IMDb URL"] = df["Name"].apply(lambda title: get_imdb_url(title, API_KEY))
        
        # Save the results to a new CSV file
        df.to_csv(OUTPUT_FILE, index=False)
        print(f"IMDb URLs saved to {OUTPUT_FILE}")
except FileNotFoundError:
    print(f"Error: File '{INPUT_FILE}' not found.")


IMDb URLs saved to imdb_urls.csv


In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Load the CSV containing IMDb links
csv_file = "C:/Users/nerea/Documents/MasterDTU/SocialGraphs_fall24/Projects/socialGraphs_fall24/FinalProject/imdb_urls.csv"  # Change this to your file path
df = pd.read_csv(csv_file)

options = webdriver.ChromeOptions()
options.add_argument("--lang=en")

driver = webdriver.Chrome(options=options)

# Function to scroll down the page to make the "183 más" button appear
def scroll_to_2_3_page(driver):
    # Scroll down halfway through the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight * 2/5);")
    time.sleep(3)  # Wait for the page to load content

def scroll_to_3_5_page(driver):
    # Scroll down halfway through the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight * 3/5);")
    time.sleep(3)  # Wait for the page to load content

# Function to click the "183 más" button and reveal all keywords
def load_keywords_for_movie(driver):
    try:
        # Scroll down the page to make sure the "183 más" button is visible
        scroll_to_2_3_page(driver)

        # Wait for the "183 más" button to appear and click it
        more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "a.sc-142fb3e4-1.cTQlEa.ipc-chip.ipc-chip--on-base"))
        )
        more_button.click()
        time.sleep(2)  # Allow the page to load

        scroll_to_3_5_page(driver)

        # Wait for the "Todo" button to appear and click it
        see_more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button.ipc-see-more__button"))
        )
        see_more_button.click()
        time.sleep(2) # Allow the page to load fully

    except Exception as e:
        print(f"Error while loading keywords for the movie: {e}")
        return False

    return True

# Function to scrape each IMDb page and extract keywords
def scrape_imdb_page(url):
    driver.get(url)
    
    # Wait for the "183 más" button and click to load keywords
    try:
        if not load_keywords_for_movie(driver):
            return None  # If we can't load the keywords, return None

        # After loading, extract all the words/keywords
        words_elements = driver.find_elements(By.CSS_SELECTOR, "a.ipc-metadata-list-summary-item__t")
        
        # Extract the text content from each word element
        words = [word.text for word in words_elements]
        
        # Join the words into a single string, separated by commas
        keywords_string = ", ".join(words)
        
        return keywords_string
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return None

# Loop through each IMDb link in the CSV
scraped_data = []
for link in df['IMDb URL']:  # Use the correct column name here
    print(f"Scraping {link}...")
    keywords = scrape_imdb_page(link)
    if keywords:
        scraped_data.append({'url': link, 'keywords': keywords})

# Save the scraped content to a new CSV file
scraped_df = pd.DataFrame(scraped_data)
scraped_df.to_csv("scraped_imdb_keywords.csv", index=False)

# Close the driver
driver.quit()


Scraping https://www.imdb.com/title/tt0111161/...


KeyboardInterrupt: 

In [8]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Load the CSV containing IMDb links
csv_file = "C:/Users/nerea/Documents/MasterDTU/SocialGraphs_fall24/Projects/socialGraphs_fall24/FinalProject/imdb_urls.csv"  # Change this to your file path
df = pd.read_csv(csv_file)

options = webdriver.ChromeOptions()
options.add_argument("--lang=en")

driver = webdriver.Chrome(options=options)

# Function to scroll down the page to make the "183 más" button appear
def scroll_to_2_3_page(driver):
    # Scroll down halfway through the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight * 2/5);")
    time.sleep(3)  # Wait for the page to load content

def scroll_to_3_5_page(driver):
    # Scroll down halfway through the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight * 3/6);")
    time.sleep(3)  # Wait for the page to load content

# Function to click the "183 más" button and reveal all keywords
def load_keywords_for_movie(driver):
    try:
        # Scroll down the page to make sure the "183 más" button is visible
        scroll_to_2_3_page(driver)

        # Wait for the "183 más" button to appear and click it
        more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "a.sc-142fb3e4-1.cTQlEa.ipc-chip.ipc-chip--on-base"))
        )
        more_button.click()
        time.sleep(2)  # Allow the page to load

        scroll_to_3_5_page(driver)

        # Wait for the "Todo" button to appear and click it
        see_more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button.ipc-see-more__button"))
        )
        see_more_button.click()
        time.sleep(2) # Allow the page to load fully

    except Exception as e:
        print(f"Error while loading keywords for the movie: {e}")
        return False

    return True

# Function to scrape each IMDb page and extract keywords
def scrape_imdb_page(url):
    driver.get(url)
    
    # Wait for the "183 más" button and click to load keywords
    try:
        if not load_keywords_for_movie(driver):
            return None  # If we can't load the keywords, return None

        # After loading, extract all the words/keywords
        words_elements = driver.find_elements(By.CSS_SELECTOR, "a.ipc-metadata-list-summary-item__t")
        
        # Extract the text content from each word element
        words = [word.text for word in words_elements]
        
        # Join the words into a single string, separated by commas
        keywords_string = ", ".join(words)
        
        return keywords_string
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return None

# Loop through each IMDb link in the CSV
scraped_data = []
for link in df['IMDb URL']:  # Use the correct column name here
    print(f"Scraping {link}...")
    keywords = scrape_imdb_page(link)
    if keywords:
        scraped_data.append({'url': link, 'keywords': keywords})

# Save the scraped content to a new CSV file
scraped_df = pd.DataFrame(scraped_data)
scraped_df.to_csv("scraped_imdb_keywords.csv", index=False)

# Close the driver
driver.quit()


Scraping https://www.imdb.com/title/tt0111161/...
Scraping https://www.imdb.com/title/tt0068646/...
Scraping https://www.imdb.com/title/tt0468569/...
Scraping https://www.imdb.com/title/tt0071562/...
Scraping https://www.imdb.com/title/tt0108052/...
Scraping https://www.imdb.com/title/tt0167260/...
Scraping https://www.imdb.com/title/tt0050083/...
Error while loading keywords for the movie: Message: element click intercepted: Element <a class="sc-142fb3e4-1 cTQlEa ipc-chip ipc-chip--on-base" tabindex="0" aria-disabled="false" href="/title/tt0050083/keywords/?ref_=tt_stry_kw">...</a> is not clickable at point (479, 49). Other element would receive the click: <button class="ipc-scroll-to-top-button sc-87e28572-0 hJXYmZ visible ipc-chip ipc-chip--on-base" tabindex="0" aria-label="Back to top" aria-disabled="false">...</button>
  (Session info: chrome=131.0.6778.86)
Stacktrace:
	GetHandleVerifier [0x00007FF61BD96CB5+28821]
	(No symbol) [0x00007FF61BD03840]
	(No symbol) [0x00007FF61BBA578A]